In [11]:
import requests

headers = {}
URL = "https://api.thegraph.com/subgraphs/name/1hive/honey-pot-xdai"

def run_query(query, query_variables): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post(URL, json={'query': query, 'variables': query_variables}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.
query = """
query Proposals($first: Int!, $skip: Int!, $proposalTypes: [Int]!, $statuses: [Int]!, $metadata: String!, $orderBy: String!, $orderDirection: String!) {
  proposals(where: {typeInt_in: $proposalTypes, statusInt_in: $statuses, metadata_contains: $metadata}, first: $first, skip: $skip, orderBy: $orderBy, orderDirection: $orderDirection) {
    id
    number
    creator
    status
    type
    createdAt
    metadata
    link
    beneficiary
    requestedAmount
    totalTokensStaked
  }
}
"""

query_variables = {
  "first": 1000,
  "skip": 0,
  "orderBy": "number",
  "orderDirection": "asc",
  "proposalTypes": [0,1],
  "statuses": [0,1,2],
  "metadata": ""
}

result = run_query(query, query_variables) # Execute the query

In [12]:
import pandas as pd
import panel as pn
import hvplot.pandas
import datetime as dt

status
Active       0250000000000000000005000000000000000000075000...
Cancelled    5000000000000000000175000000000000000075000000...
Executed     3000000000000000000500000000000000000002000000...
Name: requestedAmount, dtype: object

In [29]:
df = pd.DataFrame(result['data']['proposals'])
df['requestedAmount'] = df['requestedAmount'].apply(lambda x: int(x) // int(1e18))

In [30]:
df.groupby('status')['requestedAmount'].describe()

,count,mean,std,min,25%,50%,75%,max
status,,,,,,,,
Active,30.0,36.566667,110.997572,0.0,0.00,0.0,23.75,600.0
Cancelled,63.0,675.714286,1869.924627,0.0,5.00,40.0,100.00,7500.0
Executed,28.0,65.642857,74.963307,0.0,11.25,40.0,84.75,300.0


In [31]:
df.sort_values('requestedAmount', ascending=False)

,beneficiary,createdAt,creator,id,link,metadata,number,requestedAmount,status,totalTokensStaked,type
100,0xea0801f6923c128a24b305c41b171a1e67f6fe38,1610456835,0xea0801f6923c128a24b305c41b171a1e67f6fe38,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/improve-mobile-walle...,test,101,7500,Cancelled,None,Proposal
108,0x142f9789182a1003230fbbae586dd25d3f201803,1610486280,0x142f9789182a1003230fbbae586dd25d3f201803,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/harberger-commons-re...,7500,109,7500,Cancelled,None,Proposal
109,0x142f9789182a1003230fbbae586dd25d3f201803,1610486355,0x142f9789182a1003230fbbae586dd25d3f201803,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/harberger-commons-re...,give me everything,110,7000,Cancelled,None,Proposal
110,0x142f9789182a1003230fbbae586dd25d3f201803,1610486580,0x142f9789182a1003230fbbae586dd25d3f201803,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/improve-mobile-walle...,eagaml,111,6500,Cancelled,None,Proposal
111,0x142f9789182a1003230fbbae586dd25d3f201803,1610486625,0x142f9789182a1003230fbbae586dd25d3f201803,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/improve-mobile-walle...,aga,112,6000,Cancelled,None,Proposal
...,...,...,...,...,...,...,...,...,...,...,...
84,0xdc9d5ba277ba2e11ac5d7e8d70fe2508b312c9c2,1606943950,0xdc9d5ba277ba2e11ac5d7e8d70fe2508b312c9c2,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/defi-times-newslette...,DEFI TIMES - Article and Podcast about HoneySwap,85,0,Active,57430713843328887,Proposal
86,0x0000000000000000000000000000000000000000,1607271685,0xb9b93bf2db3678b010e71ef701d763aee144e93e,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/hny-to-the-moon/1602,HNY to the moon!,87,0,Active,3864906071746358,Suggestion
87,0x0000000000000000000000000000000000000000,1607479900,0xb82f28b33c171498f29867568323d4eba240eb30,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/honeyswap-better-the...,Honeyswap better then uniswap for staking stak...,88,0,Active,9775683127830147,Suggestion
93,0x0000000000000000000000000000000000000000,1609085835,0xbb892287c7315a6a8ef914be958e1df995442b0c,appAddress:0x7f67f5de7b815c5709be70f5eea627de4...,https://forum.1hive.org/t/suggestion-of-an-inc...,Suggestion of an increase in incentives for li...,94,0,Cancelled,0,Suggestion


In [19]:
pn.Row(
    df.query("status == 'Executed'").groupby('beneficiary').sum()[['requestedAmount']].sort_values(by='requestedAmount', ascending=False).reset_index().hvplot.table(title="Beneficiaries"), 
    df.query("status == 'Executed'").groupby('creator').sum()[['requestedAmount']].sort_values(by='requestedAmount', ascending=False).reset_index().hvplot.table(title="Creators"),
)

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [32]:
df = pd.DataFrame(result['data']['proposals'])
df['requestedAmount'] = df['requestedAmount'].apply(lambda x: int(x) // int(1e18))
df = df[['createdAt','metadata','link','requestedAmount','status','type']].query("status == 'Executed'")
df['datetime'] = pd.to_datetime(df['createdAt'].apply(lambda x: dt.datetime.fromtimestamp(int(x))))
df

,createdAt,metadata,link,requestedAmount,status,type,datetime
2,1603330365,xDai-Faucet Development and Implementation,https://forum.1hive.org/t/xdai-faucet-developm...,3,Executed,Proposal,2020-10-21 18:32:45
5,1603347045,Recovering Burnt Honey - Stuck in Agent,https://forum.1hive.org/t/recovering-burnt-hon...,50,Executed,Proposal,2020-10-21 23:10:45
19,1603395105,Make Honey Sticky with Automatic Dividend Rein...,https://forum.1hive.org/t/proposal-make-honey-...,2,Executed,Proposal,2020-10-22 12:31:45
20,1603489070,Top up the faucet,https://forum.1hive.org/t/proposal-to-top-up-t...,123,Executed,Proposal,2020-10-23 14:37:50
24,1603682900,Monthly Reddit (r/HNY) NFT Competition,https://forum.1hive.org/t/monthly-reddit-r-hny...,1,Executed,Proposal,2020-10-25 20:28:20
30,1603826595,“Add HNY to your wallet” button on UI,https://forum.1hive.org/t/add-hny-to-your-wall...,1,Executed,Proposal,2020-10-27 12:23:15
36,1603907410,Activate 10 New Farming Pairs - WBTC-XDAI (40 ...,https://forum.1hive.org/t/liquidity-pairs-for-...,40,Executed,Proposal,2020-10-28 10:50:10
42,1603907720,Activate 10 New Farming Pairs - HNY-LINK (40 H...,https://forum.1hive.org/t/liquidity-pairs-for-...,40,Executed,Proposal,2020-10-28 10:55:20
44,1603909925,Activate 10 New Farming Pairs - WETH-WBTC (40 ...,https://forum.1hive.org/t/liquidity-pairs-for-...,40,Executed,Proposal,2020-10-28 11:32:05
45,1604009235,Compensation for Discord emojis,https://forum.1hive.org/t/compensation-for-dis...,1,Executed,Proposal,2020-10-29 15:07:15


In [34]:
df.hvplot.bar(x='metadata',y='requestedAmount')

:Bars   [metadata]   (requestedAmount)

In [29]:
df['cummulativeSpend'] = df['requestedAmount'].cumsum()
df.hvplot.line(x='datetime',y='cummulativeSpend', rot=45)

:Curve   [datetime]   (cummulativeSpend)